In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/git/BigRiverQTL.jl`


In [2]:
Pkg.instantiate()

In [3]:
using Revise

In [4]:
using BigRiverQTLPlots
using Random, Statistics
using Plots
using Helium
using BigRiverQTL
using CSV
using DataFrames
using BulkLMM

In [5]:
##############
# BXD spleen #
##############

########
# Data #
########
data_dir = joinpath(@__DIR__, "../data/BXD/");
file = joinpath(data_dir, "bxd.json");

In [6]:
# Transforming data to a optimised and accessible data type
data = get_geneticstudydata(file);

In [9]:
gInfo=data.gmap;
pInfo=data.phenocov;
pheno=data.pheno;
pheno=data.pheno.val;
geno=reduce(hcat, data.geno.val);
geno_processed=convert(Array{Float64}, geno);

In [10]:
#################
# Preprocessing #
#################
traitID = 1112;
pheno_y = pheno[:, traitID];
pheno_y2=ones(length(pheno_y));
idx_nothing = findall(x->x!=nothing,pheno_y)
pheno_y2[idx_nothing]=pheno_y[idx_nothing];

In [11]:
###########
# Kinship #
###########
kinship = kinship_gs(geno_processed,.99);

In [12]:
########
# Scan #
########

single_results_perms = scan(
	pheno_y2,
	geno_processed,
	kinship;
	permutation_test = true,
	nperms = 1000,
);

In [13]:
########
# Data #
########
bulklmmdir = dirname(pathof(BulkLMM));

gmap_file = joinpath(bulklmmdir, "..", "data", "bxdData", "gmap.csv");
gInfo2 = BulkLMM.CSV.read(gmap_file, BulkLMM.DataFrames.DataFrame);


In [14]:
function gmap2df(gmap::Gmap)
    # Locus = reduce(vcat,gmap.marker_name);
    # Mb= reduce(vcat,gmap.pos);
    start=0
    Chr =repeat(["0"],sum(length.(gmap.marker_name)))
    
    # Pos =repeat(["Missing"],sum(length.(gmap.marker_name)))
    
    for i in eachindex(gmap.chr)
        l_i=length(gmap.marker_name[i])
        Chr[start+1:l_i+start] .= gmap.chr[i]
        start=start+l_i
    end
    df=DataFrame(
        Locus = reduce(vcat,gmap.marker_name),
        Chr = Chr,
        Pos= reduce(vcat,gmap.pos)
    )
        # Cm = repeat(["Missing"],sum(length.(gmap.marker_name)))
    return df

end

gmap2df (generic function with 1 method)

In [15]:
gInfo3=gmap2df(gInfo)

Row,Locus,Chr,Pos
,String,String,Float64
1,rs31443144,1,1.499
2,rs6269442,1,1.499
3,rs32285189,1,1.62623
4,rs258367496,1,1.62623
5,rs32430919,1,1.75345
6,rs36251697,1,1.88068
7,rs30658298,1,2.00791
8,rs51852623,1,2.00791
9,rs31879829,1,2.13513


In [16]:
########
# Plot #
########

plot_QTL(single_results_perms, gInfo3, mbColname = "Pos")



UndefVarError: UndefVarError: `plot_QTL` not defined

In [23]:
gInfo

Gmap(["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "X"], [["rs31443144", "rs6269442", "rs32285189", "rs258367496", "rs32430919", "rs36251697", "rs30658298", "rs51852623", "rs31879829", "rs36742481"  …  "rs13476310", "rs31478915", "rs33174703", "rs30453157", "rs32271175", "rs13476313", "rs45997766", "rs32662997", "rs29611294", "rs29609526"], ["rs33299937", "rs13476332", "rs13476335", "Affy_4430433etal", "rs33562363", "rs13476350", "rs13476352", "rs29825025", "rs13476355", "Affy_PC2_15"  …  "rs27683255", "rs27683096", "rs27680502", "rs27680457", "rs27680311", "rs6305540", "rs33599721", "rs27654345", "rs27652608", "rs6317711"], ["rs29619692", "rs13476950", "rs30702264", "rs13476958", "rs29623223", "rs13476961", "rs30460351", "rs29659075", "rs29660738", "rs29655312"  …  "rs30703217", "rs30801216", "rs13477514", "rs13477515", "rs30916671", "rs31290414", "rs30892471", "rs30353501", "rs13477528", "rs36403089"], ["rs27666058", "rs2769

In [31]:
plot_QTL(single_results_perms, gInfo, mbColname = "Pos")

MethodError: MethodError: no method matching plot_QTL(::@NamedTuple{sigma2_e::Float64, h2_null::Float64, lod::Vector{Float64}, L_perms::Matrix{Float64}}, ::Gmap; mbColname::String)

Closest candidates are:
  plot_QTL(::NamedTuple, !Matched::DataFrame; chrColname, mbColname, significance, kwargs...)
   @ BigRiverQTLPlots ~/.julia/packages/BigRiverQTLPlots/3RIW6/src/qtl/plot_QTL.jl:49
